# U-net 2

#### Files importation

In [ ]:
!pip install albumentations --quiet
!pip install git+https://github.com/tensorflow/examples.git --quiet
!pip install tensorflow_addons --quiet
!pip install segmentation-models --quiet

In [ ]:
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
from shutil import copyfile
import numpy as np
import scipy.io
from tensorflow.keras import layers
import glob
import imageio
import os
import PIL
import time
from IPython import display
import albumentations as A
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2 as cv
import datetime
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda, Activation, MaxPool2D, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from IPython.display import clear_output
from sklearn.preprocessing import LabelEncoder
from tensorflow_examples.models.pix2pix import pix2pix
import tensorflow_addons as tfa
from keras import backend as K

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# copyfile('gdrive/My Drive/DEEP-project/Progetto_DL_finale/immagini_nuove.npy', 'immagini_nuove.npy')
# copyfile('gdrive/My Drive/DEEP-project/Progetto_DL_finale/maschere_nuove.npy', 'maschere_nuove.npy')
# copyfile('gdrive/My Drive/DEEP-project/Progetto_DL_finale/immagini2000.npy', 'immagini2000.npy')
# copyfile('gdrive/My Drive/DEEP-project/Progetto_DL_finale/maschere2000.npy', 'maschere2000.npy')
# copyfile('gdrive/My Drive/DEEP-project/Progetto_DL_finale/X_val.npy', 'X_val.npy')
# copyfile('gdrive/My Drive/DEEP-project/Progetto_DL_finale/y_val.npy', 'y_val.npy')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


'y_val.npy'

#### Train set

In [ ]:
num = 1600

In [ ]:
n_classes=19

In [ ]:
X_train = np.load('immagini2000.npy')[0:num,:,:,:]
y_train = np.array(tf.expand_dims(np.load('maschere2000.npy')[0:num,:,:],3))

#### Data augmentation

In [ ]:
n_augmentation = 980

In [ ]:
# immagini_nuove = np.load('immagini_nuove.npy')[0:n_augmentation,:,:,:]
# maschere_nuove = np.array(tf.expand_dims(np.load('maschere_nuove.npy')[0:n_augmentation,:,:],3))

In [ ]:
# X_train = np.concatenate((X_train,immagini_nuove),axis=0)
# y_train = np.concatenate((y_train,maschere_nuove),axis=0)

#### Categorizzare train set

In [ ]:
train_masks_cat = to_categorical(y_train, num_classes=n_classes)
y_train_cat = train_masks_cat.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2], n_classes))
y_train = 0
train_masks_cat = 0

#### Validation set

In [ ]:
n_classes = 19

In [ ]:
num_val = 980

In [ ]:
X_val = np.load('X_val.npy')[0:num_val,:,:,:]
X_val = X_val/X_val.max()
y_val = np.array(tf.expand_dims(np.load('y_val.npy')[0:num_val,:,:],3))

#### Categorizzare validation set

In [ ]:
train_masks_cat = to_categorical(y_val, num_classes=n_classes)
y_val_cat = train_masks_cat.reshape((y_val.shape[0], y_val.shape[1], y_val.shape[2], n_classes))
y_val = 0
train_masks_cat = 0

#### Data loader

In [ ]:
batch_size = 8

In [ ]:
class DataGenerator(keras.utils.Sequence):
  def __init__(self, x_data, y_data, batch_size):
    self.x, self.y = x_data, y_data
    self.batch_size = batch_size
    self.num_batches = np.ceil(len(x_data) / batch_size)
    self.batch_idx = np.array_split(range(len(x_data)), self.num_batches)

  def __len__(self):
    return len(self.batch_idx)

  def __getitem__(self, idx):
    batch_x = self.x[self.batch_idx[idx]]
    batch_y = self.y[self.batch_idx[idx]]
    return batch_x, batch_y

In [ ]:
train_generator = DataGenerator(X_train, y_train_cat, batch_size = batch_size)
#val_generator = DataGenerator(X_val, y_val_cat, batch_size = batch_size)

### U-net

In [ ]:
l2 = 0.01
learning_rate=0.0001

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=[256, 256, 3], include_top=False)

# Use the activations of these layers
layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',      # 4x4
]
base_model_outputs = [base_model.get_layer(name).output for name in layer_names]

# Create the feature extraction model
down_stack = tf.keras.Model(inputs=base_model.input, outputs=base_model_outputs)

down_stack.trainable = False

In [ ]:
# tf.keras.utils.plot_model(base_model, show_shapes=True, dpi=64)

In [ ]:
#tf.keras.utils.plot_model(down_stack, to_file='2a_U-net-iniziale.png',show_shapes=True, dpi=10000)

In [ ]:
up_stack = [
    pix2pix.upsample(512, 3),  # 4x4 -> 8x8
    pix2pix.upsample(256, 3),  # 8x8 -> 16x16
    pix2pix.upsample(128, 3),  # 16x16 -> 32x32
    pix2pix.upsample(64, 3),   # 32x32 -> 64x64
]

In [ ]:
def unet_model(output_channels:int):
  inputs = Input((256, 256, 6))

  s = Conv2D(3, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)

  # Downsampling through the model
  skips = down_stack(s)
  x = skips[-1]
  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = tf.keras.regularizers.L2(l2=l2)(x)
    x = concat([x, skip])

  # This is the last layer of the model
  last = tf.keras.layers.Conv2DTranspose(
      filters=output_channels, kernel_size=3, strides=2,
      padding='same', activation='softmax')  #64x64 -> 128x128

  x = last(x)

  model = Model(inputs=[inputs], outputs=[x])
  return model

In [ ]:
def get_model():
    return unet_model(output_channels=n_classes)

In [ ]:
def focal_loss(gamma=2., alpha=.25):
	def focal_loss_fixed(y_true, y_pred):
		pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
		pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
		return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1+K.epsilon())) - K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0 + K.epsilon()))
	return focal_loss_fixed

In [ ]:
model = get_model()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = learning_rate), loss=focal_loss(), metrics=[tf.keras.metrics.Recall(),
                                                          tf.keras.metrics.Precision(),
                                                          tf.keras.metrics.OneHotMeanIoU(num_classes=n_classes),
                                                          'accuracy']) 
#tfa.losses.SigmoidFocalCrossEntropy(), 'categorical_crossentropy', focal_loss
# model.summary()

In [ ]:
#tf.keras.utils.plot_model(model, to_file='2a_U-net_finale.png',show_shapes=True, dpi=10000)

In [ ]:
# model.summary(expand_nested=True)

In [ ]:
callback = keras.callbacks.EarlyStopping(monitor='train_loss', patience=5) #val_one_hot_mean_io_u

In [ ]:
def create_mask(pred_mask):
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

In [ ]:
# k = 87
# sample_image, sample_mask = X_val[k,:,:,:], create_mask(tf.expand_dims(y_val_cat[k,:,:,:],0)) 
k = 87
sample_image, sample_mask = X_train[k,:,:,:], create_mask(tf.expand_dims(y_train_cat[k,:,:,:],0)) 

In [ ]:
def display(display_list):
  plt.figure(figsize=(15, 15))

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    if i==0:
      plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
    else:
      plt.imshow(display_list[i])

    plt.axis('off')
  plt.show()

In [ ]:
def show_predictions(dataset=None, num=1):
  if dataset:
    for image, mask in dataset.take(num):
      pred_mask = model.predict(image)
      display([image[0], mask[0], create_mask(pred_mask)])
  else:
    display([sample_image[:,:,0:3], sample_mask[:,:,0], create_mask(model.predict(tf.expand_dims(sample_image,0)))[:,:,0]])

In [ ]:
class DisplayCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    clear_output(wait=True)
    show_predictions()
    print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

### Training

In [ ]:
history = model.fit(x = train_generator,
                    verbose=1, 
                    epochs=50, 
                    #validation_data = val_generator,
                    shuffle = True,
                    callbacks=[callback,DisplayCallback()]
                    )

In [ ]:
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(epochs, history.history['one_hot_mean_io_u'], 'y')
plt.plot(epochs, history.history['val_one_hot_mean_io_u'], 'r')
plt.title('Training and validation hot io_u')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot(epochs, history.history['precision'], 'y')
plt.plot(epochs, history.history['val_precision'], 'r')
plt.title('Training and validation precision')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


plt.plot(epochs, history.history['recall'], 'y',)
plt.plot(epochs, history.history['val_recall'], 'r')
plt.title('Training and validation recall')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

Download and upload of the model

In [ ]:
json_model = model.to_json()

In [ ]:
with open('U-net_2_aug.json', 'w') as json_file:
    json_file.write(json_model)

In [ ]:
model.save_weights('U-net_2_aug_weights.h5')

In [ ]:
!cp U-net_2_aug.json "gdrive/My Drive/dl-prog-pro/"

In [ ]:
!cp U-net_2_aug_weights.h5 "gdrive/My Drive/dl-prog-pro/"

In [ ]:
with open('U-net_2.json', 'r') as json_file:
    json_savedModel = json_file.read()
net = tf.keras.models.model_from_json(json_savedModel)
net.load_weights('U-net_2_weights.h5')